In [35]:
import pandas as pd
import numpy as np

from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, roc_auc_score

!pip install xgboost

from xgboost import XGBClassifier

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


In [36]:
file_path = "../data/nifty_with_regime.csv"

df = pd.read_csv(file_path, parse_dates=["datetime"])
df.set_index("datetime", inplace=True)

df.head()


,open,high,low,close,volume,ema_5,ema_15,ema_gap,returns,volatility,candle_range,candle_body,hour,minute,day_of_week,returns_lag_1,returns_lag_3,ema_gap_lag_1,regime
datetime,,,,,,,,,,,,,,,,,,,
2015-01-09 11:00:00,8285.75,8288.85,8282.30,8288.00,0,8285.789434,8286.208532,-0.419098,0.000278,0.000489,6.55,2.25,11,0,4,0.000127,0.000060,-1.268458,0
2015-01-09 11:05:00,8287.95,8288.10,8280.20,8281.55,0,8284.376289,8285.626216,-1.249927,-0.000778,0.000485,7.90,6.40,11,5,4,0.000278,0.000338,-0.419098,0
2015-01-09 11:10:00,8281.85,8286.35,8279.90,8282.35,0,8283.700860,8285.216689,-1.515829,0.000097,0.000465,6.45,0.50,11,10,4,-0.000778,0.000127,-1.249927,0
2015-01-09 11:15:00,8282.55,8283.20,8276.15,8278.45,0,8281.950573,8284.370853,-2.420280,-0.000471,0.000457,7.05,4.10,11,15,4,0.000097,0.000278,-1.515829,-1
2015-01-09 11:20:00,8278.55,8280.70,8266.55,8266.55,0,8276.817049,8282.143246,-5.326197,-0.001437,0.000549,14.15,12.00,11,20,4,-0.000471,-0.000778,-2.420280,-1


In [37]:
df["future_return"] = df["returns"].shift(-3)
df["target"] = (df["future_return"] > 0).astype(int)


In [38]:
features = [
    "ema_5", "ema_15", "ema_gap",
    "volatility",
    "returns_lag_1", "returns_lag_3",
    "hour", "minute",
    "regime"
]
df.dropna(inplace=True)

X = df[features]
y = df["target"]


MODEL A — XGBOOST

In [44]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


In [46]:
tscv = TimeSeriesSplit(n_splits=5)

xgb_model = XGBClassifier(
    n_estimators=200,
    max_depth=4,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    eval_metric="logloss",
    random_state=42
)

for train_idx, test_idx in tscv.split(X_scaled):
    X_train, X_test = X_scaled[train_idx], X_scaled[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    xgb_model.fit(X_train, y_train)


In [48]:
df["xgb_prob"] = xgb_model.predict_proba(X_scaled)[:, 1]


MODEL B — LSTM

In [51]:
sequence_length = 10

lstm_features = [
    "close", "returns", "ema_gap",
    "volatility", "regime"
]

scaler_lstm = MinMaxScaler()
scaled_data = scaler_lstm.fit_transform(df[lstm_features])


In [53]:
X_lstm, y_lstm = [], []

for i in range(sequence_length, len(scaled_data)):
    X_lstm.append(scaled_data[i-sequence_length:i])
    y_lstm.append(df["target"].iloc[i])

X_lstm = np.array(X_lstm)
y_lstm = np.array(y_lstm)


In [54]:
split = int(0.7 * len(X_lstm))

X_train, X_test = X_lstm[:split], X_lstm[split:]
y_train, y_test = y_lstm[:split], y_lstm[split:]


In [55]:
lstm_model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2])),
    Dropout(0.3),
    Dense(1, activation="sigmoid")
])

lstm_model.compile(
    loss="binary_crossentropy",
    optimizer="adam",
    metrics=["accuracy"]
)


C:\Users\Hp\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [56]:
lstm_model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=64,
    validation_data=(X_test, y_test),
    verbose=1
)


Epoch 1/10
2093/2093 ━━━━━━━━━━━━━━━━━━━━ 34s 14ms/step - accuracy: 0.5047 - loss: 0.6940 - val_accuracy: 0.5060 - val_loss: 0.6931
Epoch 2/10
2093/2093 ━━━━━━━━━━━━━━━━━━━━ 23s 11ms/step - accuracy: 0.5070 - loss: 0.6932 - val_accuracy: 0.5058 - val_loss: 0.6938
Epoch 3/10
2093/2093 ━━━━━━━━━━━━━━━━━━━━ 20s 9ms/step - accuracy: 0.5023 - loss: 0.6932 - val_accuracy: 0.5052 - val_loss: 0.6931
Epoch 4/10
2093/2093 ━━━━━━━━━━━━━━━━━━━━ 16s 8ms/step - accuracy: 0.5077 - loss: 0.6931 - val_accuracy: 0.5059 - val_loss: 0.6931
Epoch 5/10
2093/2093 ━━━━━━━━━━━━━━━━━━━━ 17s 8ms/step - accuracy: 0.5078 - loss: 0.6931 - val_accuracy: 0.5058 - val_loss: 0.6931
Epoch 6/10
2093/2093 ━━━━━━━━━━━━━━━━━━━━ 18s 9ms/step - accuracy: 0.5097 - loss: 0.6930 - val_accuracy: 0.5066 - val_loss: 0.6930
Epoch 7/10
2093/2093 ━━━━━━━━━━━━━━━━━━━━ 17s 7ms/step - accuracy: 0.5072 - loss: 0.6930 - val_accuracy: 0.5058 - val_loss: 0.6932
Epoch 8/10
2093/2093 ━━━━━━━━━━━━━━━━━━━━ 21s 10ms/step - accuracy: 0.5071 - loss

In [57]:
lstm_probs = lstm_model.predict(X_lstm).flatten()

# Align predictions
df = df.iloc[sequence_length:]
df["lstm_prob"] = lstm_probs


5980/5980 ━━━━━━━━━━━━━━━━━━━━ 17s 3ms/step


In [61]:
output_path = "../results/ml_predictions.csv"
df.to_csv(output_path)

print(f"ML predictions saved to {output_path}")


ML predictions saved to ../results/ml_predictions.csv
